In [11]:
import re
import sqlite3 
import logging

from schema import Schema

from lexupdater.config import DATABASE, OUTPUT_DIR, DIALECTS, RULES, EXEMPTIONS
from lexupdater.config.constants import rule_schema, constraint_schema
from lexupdater.db_handler import DatabaseUpdater
from lexupdater.dialect_updater import (
    construct_full_query,
    filter_list_by_list, 
    parse_rules, 
)

# Regeloppdatering: Funksjonalitetskrav 
(fra issue [`#6`](https://github.com/peresolb/lexupdater/issues/6) i github-repoet til `lexupdater`) 


* Det bør skrives ut lister med ord som matcher søkemønstrene i regelsettene i regelfilene, én liste per dialekt.
    * DatabaseUpdater
* Denne funksjonen bør kunne kalles med et argument til lexupdater.py
* Man bør kunne spesifiere at man kun vil ha lister for utvalgte dialekter
* Man trenger ikke å kunne spesifisere hvilke regler man skal skrive ut lister for i terminalen. Dersom man kun ønsker å sjekke utvalgte regler, vil det trolig være en lett sak å kommentere ut de regelsettene man ikke er interessert i før man bruker denne funksjonaliteten.


In [ ]:

def create_ruleset(ruleset_name: str, rule_dialects: list, rule_patterns: list):
    """Create a ruleset with pre-defined rules."""
    ruleset = {
        "name": ruleset_name,
        "areas": rule_dialects,
        "rules": rule_patterns,
    }
    return ruleset_schema.validate(ruleset)


def create_rule_pattern(
        rule_pattern: str, replacement: str, constraints: list
) -> dict:
    """Create a substitution rule with pre-defined constraints."""
    rule = {
        'pattern': rule_pattern,
        'repl': replacement,
        'constraints': constraints
    }
    return rule_schema.validate(rule)


def create_constraint(field: str, pattern: str, is_regex: bool) -> dict:
    """Create a constraint dictionary for a rule."""
    constraint = {
        "field": field,
        "pattern": pattern,
        "is_regex": is_regex
    }
    return constraint_schema.validate(constraint)



## Eksempel 


OBS! Dette eksempelet er ment for illustrere regelkonstruksjonsprosessen. VI kommer ikke til å rette feil i leksikonet med `lexupdater`. 

---
Tatt fra feilsøkingsdokumentet om [/M RNX0/-sekvensen](https://docs.google.com/document/d/1zEX49CwOC8oo8oaVLAB1s3L1wM3vFoTp7vkaRN1v3n4/edit?usp=sharing) (takk til Marie for et illustrativt eksempel!): 

    Vi må 
    * Endre den feilaktige trankripsjonen /M RNX0 (AX0)/ til /M AX0 RN (AX0)/.
    * Endre den feilaktige transkripsjonen /M AX0 RNX0 (AX0)/ til /M AX0 RN (AX0)/. 

---
TIPS:
Jeg anbefaler å bruke regex101.com til selve utviklingen av regex-mønstrene, og å tikke av `Flavor` "Python2.7" og `Function` "substitution" i menyen til venstre. F.eks. https://regex101.com/r/Dyhrob/1

In [12]:
# Endre den feilaktige trankripsjonen /M RNX0 (AX0)/ til /M AX0 RN (AX0)/
correction_rule1 = create_rule_pattern(
    rule_pattern=r"\bM RNX0( AX0|\b)?", 
    replacement=r"M AX0 RN\1", 
    constraints=[]
)

# Endre den feilaktige transkripsjonen /M AX0 RNX0 (AX0)/ til /M AX0 RN (AX0)/
correction_rule2 = create_rule_pattern(
    rule_pattern=r"\bM AX0 RNX0( AX0|\b)?",
    replacement=r"M AX0 RN\1",
    constraints=[]
)


#Lag et nytt regel-sett for feilrettinger
corrections_ruleset = create_ruleset(
    ruleset_name="corrections", 
    rule_dialects=["e_spoken", "e_written"],
    rule_patterns=[correction_rule1, correction_rule2]
)

In [16]:
new_rulesets = [corrections_ruleset]
filter_dialects = ["e_spoken", "e_written"]
exemptions = EXEMPTIONS
query_type = "select"
word_table="word_tmp_tbl"



In [8]:
db_obj = DatabaseUpdater(
    db=DATABASE, rulesets=new_rulesets, dialect_names=filter_dialects, word_tbl="word_tmp_tbl", exemptions=EXEMPTIONS
)

In [14]:
db_obj.select_words_matching_rules()

In [ ]:
parsed_rules = parse_rules()

queries = construct_full_query(parsed_rules,query_type, word_table)


In [ ]:
for query, values in queries: 
    results[dialect] = db_obj._cursor.execute(stmt, (r"\bM R AX0 N( AX0|\b)?", )).fetchall()


In [8]:
results = {dialect: [] for dialect in filter_dialects}
for dialect in filter_dialects:
        stmt = SELECT_WORDS_QUERY.format(
        word_table=word_table,
        dialect=dialect, 
        
    )
    
    results[dialect] = db_obj._cursor.execute(stmt, (r"\bM R AX0 N( AX0|\b)?", )).fetchall()


In [9]:
from pprint import pprint as pp

pp(results.get("e_spoken"))

[(53173, 'Tomren', 'PM|person|SUR', '', '', 59105, 'T OAH2 M R AX0 N', 1),
 (153608,
  'demrende',
  'VB',
  'AKT|PS|SIN-PLU|IND-DEF|NOM',
  '',
  174840,
  'D EH2 M R AX0 N AX0',
  2),
 (153609,
  'demrendes',
  'VB',
  'AKT|PS|SIN-PLU|IND-DEF|GEN',
  '',
  174841,
  'D EH2 M R AX0 N AX0 S',
  2),
 (210139, 'flimrende', 'JJ', '', '', 239244, 'F L IH2 M R AX0 N AX0', 1),
 (210140,
  'flimrende',
  'VB',
  'AKT|PS|SIN-PLU|IND-DEF|NOM',
  '',
  239245,
  'F L IH2 M R AX0 N AX0',
  2),
 (210141,
  'flimrendes',
  'VB',
  'AKT|PS|SIN-PLU|IND-DEF|GEN',
  '',
  239246,
  'F L IH2 M R AX0 N AX0 S',
  2),
 (227484, 'forsommrene', '', '', '', 260546, 'F OAH2 RS OAH3 M R AX0 N AX0', 1),
 (251776,
  'gasskamrene',
  'NN',
  '|||NEU',
  '',
  290507,
  'G AH1 S K AH3 M R AX0 N AX0',
  1),
 (252197,
  'gatenumrene',
  'NN',
  '|||NEU',
  '',
  290967,
  'G AA2 T AX0 N OH3 M R AX0 N AX0',
  1),
 (262708,
  'glimrende',
  'JJ',
  'SIN-PLU|DEF|NOM||POS',
  '',
  303283,
  'G L IH2 M R AX0 N AX0',
  2)

{'name': 'additions', 'areas': ['e_spoken', 'e_written'], 'rules': [{'pattern': '\\bM R AX0 N( AX0|\\b)?', 'repl': 'M AX0 RN\\1', 'constraints': []}]}


In [15]:
db_obj.close_connection()

# TODO: 
* se over issues: prioritere
* samsnakke om arkitektur
* identifisere oppgaver for å plukke opp fortløpende 

In [ ]:
# 3: Skrive ut lister med kun ordformer og transkripsjoner, tilpasset NoFA. Må spesifiseres/tydeliggjøres. forced alignment 
# 4: Utvide leksikonet med nyord 
# 5: legge til ikke-standard dialektord 
# 6: 
# 11: 